# Part 5 -- Transform Target Data

Transform into categorical variable **up/down/neutral**.

### Load lib codes:

In [1]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/ipynb


In [2]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

In [3]:
!pip install pymongo
import pymongo

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
cli = pymongo.MongoClient(host='35.163.253.8')

In [5]:
# Instantiates when you put data in 
sp500_collection = cli.stocks_db.sp500_collection
stocks_collection = cli.stocks_db.stocks_collection
cli.stocks_db.collection_names()

['sp500_collection', 'stocks_collection']

In [50]:
print(sp500_collection.count())
print(stocks_collection.count())

505
250633


In [88]:
# Getting ALL data out of a collection
sp500_list = [document for document in sp500_collection.find()]
sp500_df = pd.DataFrame(sp500_list)
# len(sp500_df['stock'].unique())
sp500_df

,_id,status,stock
0,59375144d8b1410090fd2c82,complete,MMM
1,59375145d8b1410090fd2e7d,complete,ABT
2,59375145d8b1410090fd3078,complete,ABBV
3,59375145d8b1410090fd3273,complete,ACN
4,59375146d8b1410090fd346e,complete,ATVI
5,59375147d8b1410090fd3669,complete,AYI
6,59375147d8b1410090fd3864,complete,ADBE
7,59375147d8b1410090fd3a5f,complete,AMD
8,59375148d8b1410090fd3c5a,complete,AAP
9,59375148d8b1410090fd3e55,complete,AES


In [8]:
# Getting ALL data out of a collection
stocks_list = [document for document in stocks_collection.find()]
stocks_df = pd.DataFrame(stocks_list)
stocks_df.head()

,Close,Date,High,Low,Open,Stock,Volume,_id
0,158.98,2015-06-01,159.76,158.57,159.10,MMM,1906559,59375144d8b1410090fd2a88
1,159.08,2015-06-02,159.88,157.95,158.28,MMM,2481192,59375144d8b1410090fd2a89
2,160.16,2015-06-03,160.50,158.98,159.61,MMM,2074495,59375144d8b1410090fd2a8a
3,158.35,2015-06-04,160.38,158.09,159.51,MMM,2237628,59375144d8b1410090fd2a8b
4,157.08,2015-06-05,157.97,156.97,157.50,MMM,2314771,59375144d8b1410090fd2a8c


### Use Close price as our target (for now):

In [51]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250633 entries, 0 to 250632
Data columns (total 8 columns):
Close     250633 non-null float64
Date      250633 non-null datetime64[ns]
High      250630 non-null float64
Low       250630 non-null float64
Open      250630 non-null float64
Stock     250633 non-null object
Volume    250633 non-null int64
_id       250633 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 15.3+ MB


In [87]:
len(stocks_df['Stock'].unique())

501

In [138]:
def get_change_in_Close(df):
    list_of_df = []
    for s in df['Stock'].unique():
        try_df = df.loc[df['Stock']==s]
        try_df = try_df.sort(columns='Date')
        try_df['Diff'] = try_df['Close'].diff()
        list_of_df.append(try_df)
        
    return list_of_df

In [140]:
list_of_df_diff = get_change_in_Close(stocks_df)

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """


In [143]:
list_of_df_diff[1]

,Close,Date,High,Low,Open,Stock,Volume,_id,Diff
506,48.94,2015-06-01,49.20,48.57,48.77,ABT,5055102,59375145d8b1410090fd2c83,NaN
507,48.94,2015-06-02,49.18,48.32,48.65,ABT,5138146,59375145d8b1410090fd2c84,0.00
508,48.86,2015-06-03,49.13,48.74,48.96,ABT,3309634,59375145d8b1410090fd2c85,-0.08
509,48.68,2015-06-04,49.17,48.53,48.98,ABT,4686970,59375145d8b1410090fd2c86,-0.18
510,48.45,2015-06-05,48.63,48.01,48.59,ABT,3542377,59375145d8b1410090fd2c87,-0.23
511,48.44,2015-06-08,48.56,48.20,48.37,ABT,4352984,59375145d8b1410090fd2c88,-0.01
512,48.18,2015-06-09,48.53,48.01,48.52,ABT,3402013,59375145d8b1410090fd2c89,-0.26
513,48.50,2015-06-10,48.69,48.30,48.36,ABT,5614880,59375145d8b1410090fd2c8a,0.32
514,49.06,2015-06-11,49.25,48.51,48.70,ABT,7101190,59375145d8b1410090fd2c8b,0.56
515,48.67,2015-06-12,48.96,48.64,48.91,ABT,3692146,59375145d8b1410090fd2c8c,-0.39


In [44]:
stocks_df.sort(columns='Date')

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,Close,Date,High,Low,Open,Stock,Volume,_id
0,158.98,2015-06-01,159.76,158.57,159.10,MMM,1906559,59375144d8b1410090fd2a88
173753,74.69,2015-06-01,75.12,73.96,74.77,OMC,1260847,593751a42c7311003e9fa483
173247,64.06,2015-06-01,64.57,63.19,64.23,PCAR,1498779,593751a3d8b1410090fea0ef
172741,43.79,2015-06-01,44.06,43.64,43.84,ORCL,9764305,593751a3d8b1410090fe9ef4
172235,41.31,2015-06-01,42.05,41.23,41.88,OKE,1750945,593751a2d8b1410090fe9cf9
171729,78.89,2015-06-01,79.35,77.91,78.89,OXY,5066495,593751a2d8b1410090fe9afe
171223,222.49,2015-06-01,223.18,218.82,220.80,ORLY,568537,593751a22c7311003e9fa288
170717,22.38,2015-06-01,22.60,22.13,22.27,NVDA,9316504,593751a2d8b1410090fe9903
170211,47.59,2015-06-01,48.00,47.00,47.58,NUE,1721388,593751a22c7311003e9fa08d
169705,25.14,2015-06-01,25.30,25.00,25.17,NRG,2175721,593751a1d8b1410090fe9708


In [36]:
stocks_df['Date'] = stocks_df['Date'].date()

AttributeError: 'Series' object has no attribute 'date'

In [43]:
stocks_df['Close'].diff()

0          NaN
1         0.10
2         1.08
3        -1.81
4        -1.27
5        -0.49
6         0.39
7         2.06
8         0.82
9        -1.82
10       -2.17
11        0.62
12        0.46
13        2.67
14       -0.67
15        0.73
16        0.16
17       -2.30
18       -1.62
19        1.17
20       -3.14
21        0.35
22        1.35
23       -0.27
24       -0.30
25        0.35
26       -2.72
27        1.04
28        1.20
29        1.52
          ... 
250603    1.10
250604   -0.23
250605    1.11
250606   -1.47
250607    0.69
250608   -0.52
250609   -0.63
250610    0.49
250611   -0.54
250612    0.60
250613    0.09
250614    0.08
250615    0.45
250616   -0.27
250617    0.01
250618   -0.54
250619   -0.25
250620    0.39
250621    0.12
250622   -1.67
250623    0.59
250624    0.52
250625    0.11
250626    1.13
250627   -0.64
250628    0.18
250629   -0.43
250630   -0.58
250631    0.20
250632    0.48
Name: Close, dtype: float64

#### Clean data:

In [9]:
tweets_df['text']

0        On this National Gun Violence Awareness Day, l...
1        Forever grateful for the service and sacrifice...
2        Good to see my friend Prince Harry in London t...
3        Through faith, love, and resolve the character...
4        Our hearts go out to those killed and wounded ...
5        Excited to hear from Sierra, Imani, Filiz, and...
6        Happy Mother's Day to my love and partner on t...
7        We're rolling up our sleeves again, back where...
8        Well said, Jimmy. That's exactly why we fought...
9        RT @ObamaFoundation: It’s a beautiful day on t...
10       My heart goes out to the victims and their fam...
11       Chuck Berry rolled over everyone who came befo...
12       On International Women’s Day, @MichelleObama a...
13       RT @ObamaFoundation: Courage comes in many for...
14       Humbled to be recognized by a family with a le...
15         We asked. You answered. https://t.co/mAJvko6VqR
16       Happy Valentine’s Day, @michelleobama! Almost .

In [13]:
# import re

In [21]:
# def clean_text(text):
#     text = re.sub("'", "", text)
#     text = re.sub('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', text)
#     text = re.sub('[\W]', ' ', text)
#     return text

#### BAG OF WORDS: CountVectorizer

In [21]:
!pip install spacy

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
from spacy.en import English

In [33]:
nlp = English()

In [34]:
tweets_df['text'][0]

'On this National Gun Violence Awareness Day, let your voice be heard and show your commitment to reducing gun viole… https://t.co/S7dth6Ywb0'

In [35]:
nlp(tweets_df['text'][0]).vector.shape

(300,)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [22]:
naive_vectorizer = CountVectorizer(min_df = 3)
document_term_matrix = create_document_term_matrix(naive_vectorizer, tweets_df['text'])
document_term_matrix.sample(5)
#this is a document term matrix. It just counts the # of times a word shows up:

MemoryError: 

### Do a PCA to determine number of components for SVD: